<a href="https://colab.research.google.com/github/rahelehzarei/Machine-Learning-Experiment/blob/master/COVID_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q Kaggle
from google.colab import files

In [2]:
!pip install Pillow

In [3]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahelehzarei","key":"e56e7f514c91b4244549ca268617ddac"}'}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

In [5]:
! chmod 600 /root/.kaggle/kaggle.json

In [6]:
! kaggle datasets list

ref                                                       title                                      size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  ----------------------------------------  -----  -------------------  -------------  ---------  ---------------  
hummaamqaasim/jobs-in-data                                Jobs and Salaries in Data Science          76KB  2023-12-25 20:03:32           8652        175  1.0              
sazidthe1/data-science-salaries                           Data Science Salaries 2024                 57KB  2024-01-20 16:45:27            994         29  1.0              
nelgiriyewithana/apple-quality                            Apple Quality                             170KB  2024-01-11 14:31:07           2565         65  1.0              
huzdaria/laptop-pricing                                   Laptop Pricing                              3KB  2024-01-16 15:28:48            75

In [7]:
! kaggle datasets download  -d tawsifurrahman/covid19-radiography-database

 99% 769M/778M [00:11<00:00, 120MB/s]
100% 778M/778M [00:11<00:00, 70.0MB/s]


In [8]:
! mkdir covid19-radiography-database
! unzip covid19-radiography-database.zip -d covid19-radiography-database

Streaming output truncated to the last 5000 lines.
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7921.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7922.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7923.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7924.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7925.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7926.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7927.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7928.png  
  inflating: covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/masks/Normal-7929.png  
  inflating: covid19-radiograph

In [13]:
! mkdir -p covid19-dataset/training/normal
! mkdir -p covid19-dataset/training/covid
! mkdir -p covid19-dataset/test/normal
! mkdir -p covid19-dataset/test/covid

In [16]:
import os
import numpy as np
import shutil


image_count = 0

# Iterate over all files in the directory
for filename in os.listdir('/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images'):
    if os.path.isfile(os.path.join('/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images', filename)):
        # Check if the file extension is one of the image formats
        if any(filename.lower()):
            image_count += 1

total_images = image_count

# Example usage
mean = total_images / 2  # for example, mean at the middle
std_dev = 500  # standard deviation
num_samples = int(0.2 * total_images)

print(total_images)

indices = np.random.normal(mean, std_dev, num_samples)
indices = np.round(indices).astype(int)  # Round and convert to integers
indices = np.clip(indices, 1, total_images)  # Clip values to be within range

# print(indices)

files = os.listdir('/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images')

for idx in indices:
  image_path = os.path.join('/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images', files[idx-1])
        # Copy the ith image to the destination directory
  shutil.copy(image_path, '/content/covid19-dataset/test/normal')

10192


In [17]:
all_files = os.listdir('/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images')
test_files = os.listdir('/content/covid19-dataset/test/normal')

for file in all_files:
  if file not in test_files:
    image_path = os.path.join('/content/covid19-radiography-database/COVID-19_Radiography_Dataset/Normal/images', file)
    # Copy the ith image to the destination directory
    shutil.copy(image_path, '/content/covid19-dataset/training/normal')

In [18]:
! find /content/covid19-dataset/training/covid -maxdepth 1 -type f | wc -l
! find /content/covid19-dataset/test/covid -maxdepth 1 -type f | wc -l
! find /content/covid19-dataset/training/normal -maxdepth 1 -type f | wc -l
! find /content/covid19-dataset/test/normal -maxdepth 1 -type f | wc -l

3014
602
8928
1264


In [19]:
# from sklearn.model_selection import train_test_split

# # Assuming 'df' has a 'label' column
# train, test = train_test_split(df, test_size=0.25, stratify=df['label'])

NameError: name 'df' is not defined

In [22]:
!pip install tensorflow --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.15.0
    Uninstalling tensorflow-2.15.0:
      Successfully uninstalled tensorflow-2.15.0


In [24]:
from tensorflow.keras.preprocessing import image

train_datagen = image.ImageDataGenerator()
test_datagen = image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory('/content/covid19-dataset/training',batch_size=32, class_mode="binary")
val_generator = test_datagen.flow_from_directory('/content/covid19-dataset/test',batch_size=32, class_mode="binary")

batch_x, batch_y = next(train_generator)
print(batch_x.shape)  # This will give you the shape of the image data in each batch
print(batch_y.shape)  # This will give you the shape of the labels in each batch


Found 11942 images belonging to 2 classes.
Found 1866 images belonging to 2 classes.
(32, 256, 256, 3)
(32,)


In [25]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
print('Imported Successfully!')

Imported Successfully!


## **Create a Convolutional Neural Network**

In [26]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation="relu",input_shape=(256, 256, 3)))
model.add(Conv2D(64, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(256, kernel_size=(3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(32, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam",metrics = ["accuracy"])
model.summary()

train_datagen = image.ImageDataGenerator()
test_datagen = image.ImageDataGenerator()

train_generator = train_datagen.flow_from_directory('/content/covid19-dataset/training',batch_size=16, class_mode="binary")
val_generator = test_datagen.flow_from_directory('/content/covid19-dataset/test',batch_size=16, class_mode="binary")

hist = model.fit(train_generator, epochs = 80, validation_data=val_generator)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 254, 254, 32)      896       
                                                                 
 conv2d_1 (Conv2D)           (None, 252, 252, 64)      18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 126, 126, 64)      0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 126, 126, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 124, 124, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 62, 62, 128)       0         
 g2D)                                                   

In [27]:
from google.colab import files
from google.colab import drive


model.save('simpleCNN.h5')  # This will save the model in Colab's virtual machine
files.download('simpleCNN.h5')
# drive.mount('https://drive.google.com/drive/folders/13gMWKZJukvq-ELiXLbv8EwSA9x0mjjWL')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>